In [ ]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
import os, re, csv, math, codecs
num_words=30000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_words+2,)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [ ]:
num_words=len(np.unique(np.hstack(data)))
print("Categories:", np.unique(targets))
print("Number of unique words:", num_words)

Categories: [0 1]
Number of unique words: 30000


Agregar este archivo a la carpeta de google drive

https://drive.google.com/open?id=1zgSN39uX3OWxGyqwGDypKxQRokE0_njG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173.0


In [ ]:
print("Label:", targets[0])

Label: 1
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[1]] )
print(decoded)

1646592/1641221 [==============================] - 0s 0us/step
# big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal measures the hair is big lots of boobs bounce men wear those cut tee shirts that show off their stomachs sickening that men actually wore them and the music is just # trash that plays over and over again in almost every scene there is trashy music boobs and paramedics taking away bodies and the gym still doesn't close for # all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh a

In [ ]:
#load embeddings
EMBEDDING_DIR = "/content/drive/My Drive/ITBA/Lab-NLP/Embeddings/fasttext/"
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open(EMBEDDING_DIR+'wiki-news-300d-1M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...
found 999995 word vectors


In [ ]:
embeddings_index["car"]

array([-1.600e-02, -3.000e-04, -1.684e-01,  8.990e-02, -2.000e-02,
       -9.300e-03,  4.820e-02, -3.080e-02, -4.510e-02,  6.000e-04,
        1.680e-01,  9.650e-02,  3.061e-01, -4.110e-02,  2.960e-02,
       -4.630e-02,  3.250e-02, -7.030e-02,  2.220e-02, -1.404e-01,
       -2.638e-01, -1.340e-02,  1.277e-01,  1.227e-01,  1.803e-01,
       -1.920e-02,  3.530e-02,  1.214e-01,  1.509e-01, -8.610e-02,
        9.760e-02, -2.550e-02, -2.760e-02, -1.556e-01, -7.390e-02,
        5.430e-02, -6.700e-02, -3.000e-03,  1.515e-01,  6.080e-02,
        3.300e-02,  7.470e-02,  9.000e-04,  5.500e-02,  4.800e-03,
       -1.320e-02, -2.620e-02, -1.804e-01,  8.050e-02,  4.640e-02,
       -1.590e-02, -3.020e-02, -6.785e-01,  1.632e-01,  1.030e-02,
        6.550e-02, -8.430e-02,  2.270e-02,  3.350e-02, -3.560e-02,
       -6.380e-02, -1.111e-01, -1.700e-03,  9.780e-02,  5.650e-02,
       -3.520e-02,  3.950e-02,  1.867e-01,  7.900e-02, -1.234e-01,
        1.860e-02,  8.900e-02,  1.631e-01,  7.830e-02,  5.610e

In [ ]:
embed_dim=300
embedding_matrix=np.zeros([num_words+4,embed_dim])
for word, idx in index.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx+3,:]=embeddings_index[word]

In [ ]:
maxlen=1000

In [ ]:
data = pad_sequences(data,maxlen=maxlen)

In [ ]:
len(data[0])

1000

In [ ]:
len(data[1])

1000

In [ ]:
data=np.array(data)

In [ ]:
data.shape

(50000, 1000)

In [ ]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense
from keras.models import Sequential
from keras import optimizers
nb_words=num_words+4
num_filters=64
model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters*2, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 300)         9001200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          134464    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 500, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 128)          57472     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
______________________________

In [ ]:
model.fit(data,targets,batch_size=32,epochs=20,validation_split=0.2)



Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 34s 851us/step - loss: 0.3868 - acc: 0.8236 - val_loss: 0.2666 - val_acc: 0.8876
Epoch 2/20
40000/40000 [==============================] - 30s 758us/step - loss: 0.2732 - acc: 0.8872 - val_loss: 0.2487 - val_acc: 0.8994
Epoch 3/20
40000/40000 [==============================] - 30s 754us/step - loss: 0.2255 - acc: 0.9096 - val_loss: 0.2420 - val_acc: 0.9000
Epoch 4/20
40000/40000 [==============================] - 30s 756us/step - loss: 0.1846 - acc: 0.9272 - val_loss: 0.2429 - val_acc: 0.9012
Epoch 5/20
40000/40000 [==============================] - 30s 757us/step - loss: 0.1577 - acc: 0.9392 - val_loss: 0.2783 - val_acc: 0.8908
Epoch 6/20
40000/40000 [==============================] - 30s 754us/step - loss: 0.1271 - acc: 0.9524 - val_loss: 0.2612 - val_acc: 0.9004
Epoch 7/20
40000/40000 [==============================] - 30s 759us/step - loss: 0.1036 - acc: 0.9617 - val_loss: 

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08007812,  0.10498047,  0.04980469, ...,  0.00366211,
         0.04760742, -0.06884766],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.13378906,  0.1640625 , -0.17382812, ..., -0.25585938,
        -0.11669922,  0.08984375],
       [-0.12353516,  0.21777344, -0.578125  , ...,  0.12695312,
         0.20507812, -0.00543213],
       [-0.2265625 , -0.05151367,  0.14941406, ..., -0.19238281,
         0.01190186, -0.06542969]])